In [26]:
import datetime

import fasttext
import keras_core as keras
import numpy as np
import pandas as pd
from keras_core import Sequential
from keras_core.src.layers import Dense
from keras_core.models import load_model
from sklearn.model_selection import train_test_split

## import fastTest model

In [4]:
ft = fasttext.load_model('../../data/embedding_data/cc.fr.300.bin')

## Configurations

In [23]:
data_folder_src = '../../data/processed/'
data_path_src = f'{data_folder_src}corpus_with_text_label_final.csv'

date_str = datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
log_dir = f"../../log/fit/fasttext_nlp_{date_str}"

model_weight_dst = f'../../model_weights/fasttext_nlp_{date_str}-model.keras'

## loading dataset

In [8]:

ds = pd.read_csv(data_path_src).iloc[:, 1:].dropna()
ds

,full_text,label
1,@MamzelleMNa Une très humble femme! #Ironie,0
2,"BLOGUE - «Tsé, la parité homme-femme...» au se...",0
5,Mise en ligne de mon article sur @PayeTaShnek ...,0
7,Hommage #journeedelafemme Toute sa vie elle a ...,0
8,Lustre bois flotté feuilles exotiques bleues a...,0
...,...,...
9545,"[Exclusif] #BalanceTonPorc, transparence, juge...",1
9547,"Eric Monier, directeur de la rédaction de #LCI...",1
9548,.@annesaurat porte plainte contre son ancien r...,1
9550,"Moïra Sauvage @excisionparlons ""une pensée pou...",1


## text to vector

In [9]:
def text_to_vector(text):
    words = text.split(' ')
    word_vectors = [ft.get_word_vector(word) for word in words if word in ft.words]
    if not word_vectors:
        return np.zeros(300)
    return np.mean(word_vectors, axis=0)

In [10]:
ds['vector'] = ds['full_text'].apply(text_to_vector)

X = np.vstack(ds['vector'])
y = ds['label'].values

## building and training the model

In [11]:
experiment_name = 'deep_woke_fasttext_classification_mlp'

### callbacks

In [12]:
early_stopping = keras.callbacks.EarlyStopping(
    patience=5,
    min_delta=0.001,
    restore_best_weights=True,
    monitor='loss'
)


In [13]:
tensorboard_callback = keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)

### training the model

In [14]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

model = Sequential()
model.add(Dense(128, input_dim=300, activation='relu'))
model.add(Dense(64, activation='relu'))
model.add(Dense(1, activation='sigmoid'))

model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

model.fit(X_train,
          y_train,
          epochs=500,
          batch_size=100,
          callbacks=[
              early_stopping,
              tensorboard_callback,
          ],
          validation_data=(X_test, y_test)
          )

/home/arys/miniconda3/envs/DEV_IA/lib/python3.10/site-packages/keras_core/src/layers/core/dense.py:73: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
2024-06-13 12:02:39.471532: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:984] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-06-13 12:02:39.729046: W tensorflow/core/common_runtime/gpu/gpu_device.cc:2251] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required libraries for your platform.
Skipping registering GPU devices...


Epoch 1/500


I0000 00:00:1718272964.744891   54211 service.cc:145] XLA service 0x7fc24000ad50 initialized for platform Host (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1718272964.746184   54211 service.cc:153]   StreamExecutor device (0): Host, Default Version
2024-06-13 12:02:44.853215: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:268] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
I0000 00:00:1718272965.359417   54211 device_compiler.h:188] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


26/26 ━━━━━━━━━━━━━━━━━━━━ 6s 26ms/step - accuracy: 0.7671 - loss: 0.6116 - val_accuracy: 0.8812 - val_loss: 0.3745
Epoch 2/500
26/26 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.8553 - loss: 0.4240 - val_accuracy: 0.8812 - val_loss: 0.3593
Epoch 3/500
26/26 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.8563 - loss: 0.3957 - val_accuracy: 0.8812 - val_loss: 0.3455
Epoch 4/500
26/26 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.8500 - loss: 0.3870 - val_accuracy: 0.8812 - val_loss: 0.3365
Epoch 5/500
26/26 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.8585 - loss: 0.3628 - val_accuracy: 0.8812 - val_loss: 0.3319
Epoch 6/500
26/26 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.8534 - loss: 0.3594 - val_accuracy: 0.8812 - val_loss: 0.3301
Epoch 7/500
26/26 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.8542 - loss: 0.3537 - val_accuracy: 0.8812 - val_loss: 0.3285
Epoch 8/500
26/26 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.8577 - loss: 0.3526 - val_accuracy: 0.8812 - val_loss: 0.

# save model

In [24]:
model.save(model_weight_dst)

### testing with custom sentence

In [27]:
new_model = load_model(model_weight_dst)

/home/arys/miniconda3/envs/DEV_IA/lib/python3.10/site-packages/keras_core/src/saving/saving_lib.py:347: UserWarning: Skipping variable loading for optimizer 'rmsprop', because it has 8 variables whereas the saved optimizer has 14 variables. 
  trackable.load_own_variables(weights_store.get(inner_path))


In [28]:
text_nice = [
    "la bonjour je suis un type sympas", 
    "hello ça va les gens",
    "j'aime pas trop ce mec",
    "cette meuf est cool",
    "les meuf sont trop cool",
    "les mecs sont des gens stylé",
    "le mec est stylé",
    "nan mais mon manager exagère quand il dit que je suis pas assez productif et que je travail pas assez, franchement il abuse",
    "les hommes cis sont toxiques pour la société car ils imposent leur vision paternaliste",
    "Les hommes sont moins matures que les femmes quand il s'agit de prendre des décisions"
    
]

text_no_nice = [
    "regarde moi cette connasse",
    "les femme leur place c'est à la cuisine",
    "femme au volant, accident",
    "les femmes on peux pas leur faire confiance",
    "Pour une transphobe son passing est pas ouf",
    "Elle l’ouvre quand même beaucoup celle là pour quelqu’un qu’est pas capable d’éduquer ses enfants",
    "Dis donc, c’est pas parce que t’es mal baisée que tu peux tout te permettre ????",
    "Moi ce que j’aime bien chez les filles comme toi c’est que t’es pas du genre prise de tête",
    "les femmes c'est comme le fromage, quand elles puent c'est qu'elles sont bonnes",
    "Oh les grosses tepus",
    "Faut pas t'étonner si tu te fais violer vu comment t'es habillée",
    "Tu deverais avoir des enfants maintenant avant que ce soit trop tard",
    "Vous seriez une collaboratrice idéal pour notre entreprise, malheureusement nous cherchons quelqu’un disponible sur le long terme, et nous craignons qu’à votre âge des changements familiaux ne vous permettent plus de vous investir autant que besoins"
]
wordsvec_nice = [np.expand_dims(text_to_vector(text), axis=0) for text in text_nice]
wordsvec_no_nice = [np.expand_dims(text_to_vector(text), axis=0) for text in text_no_nice]

print("nice output")
for i in range(0, len(wordsvec_nice)):
    prediction = new_model.predict(wordsvec_nice[i], verbose=False)
    print(f"{text_nice[i]} : {prediction > 0.0001}")


print("\n\nnot nice output")
for i in range(0, len(wordsvec_no_nice)):
    prediction =  new_model.predict(wordsvec_no_nice[i], verbose=False)
    print(f"{text_no_nice[i]} : {prediction > 0.0001}")

nice output
la bonjour je suis un type sympas : [[False]]
hello ça va les gens : [[ True]]
j'aime pas trop ce mec : [[ True]]
cette meuf est cool : [[ True]]
les meuf sont trop cool : [[ True]]
les mecs sont des gens stylé : [[ True]]
le mec est stylé : [[ True]]
nan mais mon manager exagère quand il dit que je suis pas assez productif et que je travail pas assez, franchement il abuse : [[False]]
les hommes cis sont toxiques pour la société car ils imposent leur vision paternaliste : [[ True]]
Les hommes sont moins matures que les femmes quand il s'agit de prendre des décisions : [[ True]]


not nice output
regarde moi cette connasse : [[ True]]
les femme leur place c'est à la cuisine : [[ True]]
femme au volant, accident : [[ True]]
les femmes on peux pas leur faire confiance : [[ True]]
Pour une transphobe son passing est pas ouf : [[False]]
Elle l’ouvre quand même beaucoup celle là pour quelqu’un qu’est pas capable d’éduquer ses enfants : [[ True]]
Dis donc, c’est pas parce que t’es